In [2]:
import pandas as pd

In [3]:
t = pd.read_csv('/home/jovyan/python-seminar/Homeworks/hw_8/location_data_hw8_2022.csv')

In [4]:
t

,t,red_pos_X,red_pos_Y,blue_pos_X,blue_pos_Y
0,1.718397,1.330174,0.993564,1.081700,-1.295918
1,2.998056,1.310404,1.012772,1.004997,-1.254394
2,6.015073,1.270312,1.028386,1.113696,-1.321257
3,8.267651,1.331131,0.948223,1.001845,-1.285209
4,9.279321,1.346159,0.963485,1.113008,-1.261147
...,...,...,...,...,...
233,917.257513,1.508631,-1.102589,1.336283,0.731539
234,918.731229,1.527784,-1.089354,1.365426,0.704495
235,920.799970,1.548625,-1.114348,1.274398,0.851536
236,935.253542,1.546141,-1.122522,1.375008,0.789581


In [8]:
import numpy as np 

px_red = np.polyfit(t['t'], t['red_pos_X'], 1)
py_red = np.polyfit(t['t'], t['red_pos_Y'], 1)
px_blue = np.polyfit(t['t'], t['blue_pos_X'], 1)
py_blue = np.polyfit(t['t'], t['blue_pos_Y'], 1)

In [9]:
print(px_red)
print(py_red)
print(px_blue)
print(py_blue)

[2.58402337e-04 1.30230585e+00]
[-0.00226272  0.98191362]
[2.70710365e-04 1.09354086e+00]
[ 0.00226952 -1.30433238]


In [14]:
import pymc3 as pm

with pm.Model() as model:
    # Define priors.
    sigma = pm.HalfCauchy("sigma", beta=10, testval=1.0)
    mx_red = pm.Normal("mx_red", mu=px_red[0], sigma=1)
    bx_red = pm.Normal("bx_red", mu=px_red[1], sigma=1)
    mx_blue = pm.Normal("mx_blue", mu=px_blue[0], sigma=1)
    bx_blue = pm.Normal("bx_blue", mu=px_blue[1], sigma=1)
    my_red = pm.Normal("my_red", mu=py_red[0], sigma=1)
    by_red = pm.Normal("by_red", mu=py_red[1], sigma=1)
    my_blue = pm.Normal("by_blue", mu=py_blue[0], sigma=1)
    by_blue = pm.Normal("bb_blue", mu=py_blue[1], sigma=1)

    likelihood = pm.Normal("xred", mu=bx_red + mx_red*t['t'], sigma=sigma, observed=t['red_pos_X']) + \
                 pm.Normal("yred", mu=by_red + my_red*t['t'], sigma=sigma, observed=t['red_pos_Y']) + \
                 pm.Normal("xblue", mu=bx_blue + mx_blue*t['t'], sigma=sigma, observed=t['blue_pos_X']) + \
                 pm.Normal("yblue", mu=by_blue + my_blue*t['t'], sigma=sigma, observed=t['blue_pos_Y'])

    # Inference!
    # draw 3000 posterior samples using NUTS sampling
    trace = pm.sample(3000, return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [bb_blue, by_blue, by_red, my_red, bx_blue, mx_blue, bx_red, mx_red, sigma]


Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 41 seconds.


RuntimeError: 
Could not import 'mkl'.  If you are using conda, update the numpy
packages to the latest build otherwise, set MKL_THREADING_LAYER=GNU in
your environment for MKL 2018.

If you have MKL 2017 install and are not in a conda environment you
can set the Aesara flag blas__check_openmp to False.  Be warned that if
you set this flag and don't set the appropriate environment or make
sure you have the right version you *will* get wrong results.
